In [1]:
from numpy.core.numeric import NaN
from MCtool.RFilter import gray
from genericpath import exists
from matplotlib import image
import math
import sys
import time

import cv2
from matplotlib import pyplot as plt
from tensorflow.python.keras.backend import dtype
from DeepLearning import LearnAndTest
from Rpkg.Rfund.InputFeature import InputFeature
import datetime
import os
import gc
import tensorflow as tf
import random
import numpy as np
import pandas as pd

from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

import Filtering

import torch


import DeepLearning
from tensorflow.keras.optimizers import Adam

from Rpkg.Rfund.InputFeature import InputFeature
from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

from MCtool import RFilter, resultEval
from DeepLearning import save_eval_result

import numpy as np
import cv2
import torch
from transformations import ComposeDouble, FunctionWrapperDouble, create_dense_target, normalize_01
from customdatasets import SegmentationDataSet1
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib
from skimage.transform import resize

#early stopping なし
from unet import UNet
from trainer import Trainer




2025-05-26 13:51:32.330279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-26 13:51:32.994282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# #############################
# ###bugged/ doesnt read by order
# ###Custom Function renames all files in format N1-...
# ###Only need to run it one time
# #############################

# import os
# from PIL import Image

# def rename_images(directory):
#     # Get a list of all files in the directory
#     files = os.listdir(directory)
    
#     # Filter out only the image files
#     image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
#     # Sort the image files to ensure consistent naming
#     image_files.sort()
    
#     # Initialize counter
#     counter = 1
    
#     # Iterate through the image files
#     for filename in image_files:
#         # Open the image
#         image_path = os.path.join(directory, filename)
#         with Image.open(image_path) as img:
#             # Rename the image file
#             new_filename = f"N1-{counter}" + os.path.splitext(filename)[1]
#             new_image_path = os.path.join(directory, new_filename)
            
#             # Save the image with the new name
#             img.save(new_image_path)
        
#         # Increment the counter
#         counter += 1

# # Specify the directory containing the images
# directory_path = "/home/eric/Downloads/imgNaming/processing"

# # Call the function to rename the images
# rename_images(directory_path)


In [2]:
# Make sure there is no randomness in the output so that the output is reproduceable
import torch
import numpy as np
import random

# Set seed for Python random module
random.seed(42)

# Set seed for NumPy
np.random.seed(42)

# Set seed for PyTorch
torch.manual_seed(42)

# If you are using GPU
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Make the convolution operations deterministic
torch.backends.cudnn.deterministic = True

# Disable the CUDNN benchmark to ensure deterministic results
torch.backends.cudnn.benchmark = False


In [3]:
N_BLOCK = 4
LR = 0.01
OUTPUT_DIR = 666
IN_CHANNEL =108

AUGMENTATION  =  50


def random_test_choose(test_data):
    for i in range(len(test_data)):
        test_data[i] +=  '-' + str(random.randint(1, AUGMENTATION)) + '.'
    return test_data



# INPUT = ['N1', 'N2']
# VALIDATION = [ 'N3']


INPUT = ['N1', 'N2']
VALIDATION = [ 'N3']
ANNEALING = []

# on data aug
#TEST = ['oN4']

# on original dataset
TEST = ['oN4']

# Randomly chooses one image from the test augmented image to ensure uniqueness and integrity of the test dataset
# For example if the augmentaion scale is 10x, it means there are 10 aug images from single original image
# Since we cant include all the image to the testing because the original one image is used 10 times
# So random one image is chosen from the aug 10x imgs
#TEST = ['N4-1', 'N4-2', 'N4-3', 'N4-4', 'N4-5', 'N4-6', 'N4-7', 'N4-8', 'N4-9']
#TEST = random_test_choose(TEST)




print('Test dataset:')
print(TEST)

Test dataset:
['oN4']


In [4]:
# 自分の環境設定がうまくいったかどうかを確認しましょう、特にGPUの動作
# Prints the version of PyTorch installed
print('PyTorch Version installed: ' + torch.__version__)

# Prints the version of CUDA associated with the installed PyTorch version
print('CUDA version associated with PyTorch version: ' + torch.version.cuda)

# Prints the version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch
print('Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch' + str(torch.backends.cudnn.version()))

# Same as the line above
print('CUDA is available: ' + str(torch.cuda.is_available()))

# Returns the number of available CUDA-enabled GPUs
print('Number of GPUs compatible with CUDA:' + str(torch.cuda.device_count()))

# Returns the name of the GPU at index 0
print('Name of the GPU at index 0: '  + str(torch.cuda.get_device_name(0)))

# Returns the index of the current CUDA device being used
print('Current CUDA device index: '  + str(torch.cuda.current_device()))


PyTorch Version installed: 2.3.0+cu121
CUDA version associated with PyTorch version: 12.1
Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch8902
CUDA is available: True
Number of GPUs compatible with CUDA:1
Name of the GPU at index 0: NVIDIA GeForce RTX 2080 Ti
Current CUDA device index: 0


In [5]:
# ファイル名の先頭部分（prefix）により自動的にファイル名を抽出するアルゴリズム。
# 実際それぞれのファイル名は違うと思うので、必須ではない

""" 

Extracts filenames in directory if they start with the prefix input 


Args/Parameters:

    directory_path (string): The path of the dir (ex: /root/home/Documents/etc)
    
    prefix (string): Prefix of the file name (ex: 'Bo' is a prefix of 'Bone')

Returns:

    sorted_file_names (list of str): File names sorted in ascending order in the dir without extension ex: ['bone1', 'bone2', ...]

Raises:

    SomeError: ...

"""

def file_names_with_prefix(directory_path, prefix):

    # Initialize an empty list to store the file names without extensions
    file_names_without_extension = []

    # Loop through all files in the specified directory
    for filename in os.listdir(directory_path):
        #Checking if the file in loop exists in the directory_path not sure how is this necessary
        #??
        if os.path.isfile(os.path.join(directory_path, filename)):
            # Check if the file name starts with the specified prefix
            if filename.startswith(prefix):
                # Get the file name without extension
                name_without_extension, _ = os.path.splitext(filename)

                # Append the file name (without extension) to the list
                file_names_without_extension.append(name_without_extension)

    # Sort the list of file names without extensions in ascending order
    sorted_file_names = sorted(
        file_names_without_extension,
        key=lambda x: (x.split('-')[0], int(x.split('-')[1]))
    )  # Modify this part based on your file naming convention

    # Now you have a sorted list of file names with the specified prefix and without extensions
    return sorted_file_names


In [6]:
# #############################
# ###bugged/ doesnt read by order
# ###Custom Function renames all files in format N1-...
# ###Only need to run it one time
# #############################

# import os
# from PIL import Image

# def rename_images(directory):
#     # Get a list of all files in the directory
#     files = os.listdir(directory)
    
#     # Filter out only the image files
#     image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
#     # Sort the image files to ensure consistent naming
#     image_files.sort()
    
#     # Initialize counter
#     counter = 1
    
#     # Iterate through the image files
#     for filename in image_files:
#         # Open the image
#         image_path = os.path.join(directory, filename)
#         with Image.open(image_path) as img:
#             # Rename the image file
#             new_filename = f"N1-{counter}" + os.path.splitext(filename)[1]
#             new_image_path = os.path.join(directory, new_filename)
            
#             # Save the image with the new name
#             img.save(new_image_path)
        
#         # Increment the counter
#         counter += 1

# # Specify the directory containing the images
# directory_path = "/home/eric/Downloads/imgNaming/processing"

# # Call the function to rename the images
# rename_images(directory_path)


In [8]:
import pathlib
from pathlib import Path

# ここで、folder名とかPathとか色々設定

# Setting the directory name, path and other settings

# Define the root directory where your project is located
# Defining a Path object for the project's root dir
root_dir = Path(pathlib.Path.cwd())

# result folder name
date_str = '20241128-Conv1x1-' + str(OUTPUT_DIR)

# Define the directories for different types of data
# Concatenating the root dir to the different dataset dirs
data_dir = str(root_dir / "img_1006t/mcOriginal")
feature_dir = str(root_dir / "img_1006t/mcFeature") 
labeled_dir = str(root_dir / "img_1006t/labeled")

# test_data_dir = str(root_dir / "img_1006/originalTest") 
# test_labeled_dir = str(root_dir / "img_1006/testLabel") 
# test_feature_dir = str(root_dir / "img_1006/testFeature")


# annealing_img_dir = str(root_dir / "img_1006/annealing_img") # 焼きなまし法時に使う
# annealing later, original for now
annealing_img_dir = str(root_dir / "img_1006t/original")
result_dir = str(root_dir / "result" / date_str)
test_result_dir= str(root_dir / "result_test" / date_str)

# Making directories based on the path string result_dir and test_result_dir
Path(result_dir).mkdir(parents=True, exist_ok=True)
Path(test_result_dir).mkdir(parents=True, exist_ok=True)

# Prints the paths of the dirs
print('Root directory: ' + str(root_dir))
print('Data directory (original dir): ' + str(data_dir))
print('Feature img directory: ' + str(feature_dir))
print('Labeled img directory: ' + str(labeled_dir))
print('Annealing directory: ' + str(annealing_img_dir))
print('Result directory: ' + str(result_dir))
print('Test result directory: ' + str(test_result_dir))

# Defining variables filename list of path str starts with the prefix format
# In this case: N1 and N3 is training data and N2 is validation data and N4 is a test data
input_train = []
input_name_val = []
annealing_input_name = []
input_train = []
test_input_name = []
for raw_input_img in INPUT:
    input_train.extend(file_names_with_prefix(data_dir, raw_input_img))
for raw_val_img in VALIDATION:
    input_name_val.extend(file_names_with_prefix(data_dir, raw_val_img))
for raw_anneal in ANNEALING:
    annealing_input_name.extend(file_names_with_prefix(data_dir, raw_anneal))
for raw_test in TEST:
    test_input_name.extend(file_names_with_prefix(data_dir, raw_test))


####old version of assigning
# input_train = file_names_with_prefix(data_dir, INPUT)
# input_name_val = file_names_with_prefix(data_dir, VALIDATION)
# annealing_input_name = file_names_with_prefix(data_dir, ANNEALING)
# test_input_name = file_names_with_prefix(data_dir, TEST) 




# added this part to investigate the overfitting comment out when it is done
#test_input_name = input_train
#test_input_name = input_name_val


# extra_dataset = file_names_with_prefix(data_dir,'N5-')
# input_train.extend(extra_dataset)

# Prints the each data image name
print(input_train)
print(input_name_val)
print(annealing_input_name)
print(test_input_name)
# print(extra_dataset)


# Defining a var to store each list length
len_train = len(input_train)
len_val = len(input_name_val)
len_test = len(test_input_name)
len_annealing = len(annealing_input_name)


print(len(input_train))

print(len(input_name_val))
print(len(test_input_name))
print(len(annealing_input_name))


Root directory: /home/eric/Documents/cervicalResearchIIP
Data directory (original dir): /home/eric/Documents/cervicalResearchIIP/img_1006t/mcOriginal
Feature img directory: /home/eric/Documents/cervicalResearchIIP/img_1006t/mcFeature
Labeled img directory: /home/eric/Documents/cervicalResearchIIP/img_1006t/labeled
Annealing directory: /home/eric/Documents/cervicalResearchIIP/img_1006t/original
Result directory: /home/eric/Documents/cervicalResearchIIP/result/20241128-Conv1x1-666
Test result directory: /home/eric/Documents/cervicalResearchIIP/result_test/20241128-Conv1x1-666
['N1-1', 'N1-2', 'N1-3', 'N1-4', 'N1-5', 'N1-6', 'N1-7', 'N1-8', 'N1-9', 'N1-10', 'N2-1', 'N2-2', 'N2-3', 'N2-4', 'N2-5', 'N2-6', 'N2-7', 'N2-8', 'N2-9', 'N2-10']
['N3-1', 'N3-2', 'N3-3', 'N3-4', 'N3-5', 'N3-6', 'N3-7', 'N3-8', 'N3-9', 'N3-10']
[]
[]
20
10
0
0


In [9]:
# 特徴画像の特徴一覧をリストとして取得
inputfeature_list = list(map(str, InputFeature))
print(inputfeature_list)

feature_num = len(inputfeature_list)
print(feature_num)

['GRY_', 'NML1', 'NML2', 'NML3', 'TOP1', 'TOP2', 'TOP3', 'TOP4', 'SBLX', 'SBLY', 'SBLM', 'SBLD', 'SBL1', 'SBL2', 'SBL3', 'SBL4', 'LPL1', 'LPL2', 'MEA1', 'MEA2', 'GAU1', 'GAU2', 'MED1', 'MED2', 'LBP1', 'LBP2', 'LBP3', 'ETC1', 'ETC2', 'STC1', 'STC2', 'HGF_', 'NGP_', 'POS1', 'POS2', 'POS3', 'SOL_', 'EMB1', 'EMB2', 'EMB3', 'KNN1', 'KNN2', 'BLT1', 'BLT2', 'OOO_', 'CAN1', 'CAN2', 'CAN3', 'PRE1', 'PRE2', 'PRE3', 'PRE4', 'UNS1', 'UNS2', 'UNS3', 'UNS4', 'UNS5', 'FOU1', 'FOU2', 'FOU3', 'FOU4', 'ERO1', 'ERO2', 'ERO3', 'ERO4', 'ERO5', 'ERO6', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5', 'CLO1', 'CLO2', 'CLO3', 'CLO4', 'CLO5', 'SCH1', 'SCH2', 'SCH3', 'SCH4', 'ROB1', 'ROB2', 'ROB3', 'ROB4', 'MIN1', 'MIN2', 'MIN3', 'MIN4', 'MAX1', 'MAX2', 'MAX3', 'MAX4', 'MRG1', 'MRG2', 'MRG3', 'MRG4', 'MRL1', 'MRL2', 'MRL3', 'MRL4', 'BTM1', 'BTM2', 'BTM3', 'BTM4', 'DST_', 'HOM_', 'RIC_']
108


In [10]:
## 重み計算なし
def CreateWeightImage(input_number):
    print("in create now")
    label_dataset = []
    arrDataset = []
    for i in input_number:
        label_path = os.path.join(labeled_dir, f"{i}.png")
        input_originallabel = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        # commented the binary label because the project has more labels than 2 
        #_, binary_label = cv2.threshold(input_originallabel, 0, 255, cv2.THRESH_BINARY)
        
        ## This part was used for decreasing and increasing the label count when there was inconsistency with the label dataset
        
        # if len(np.unique(input_originallabel)) > 11:
        #     print("Defected image detected (more labels):" + f"{i}.png")
        #     print(np.unique(input_originallabel))
        #     for y in range(256):
        #         for x in range (256):
        #             if (input_originallabel[y][x] == 11) or (input_originallabel[y][x] == 12):
        #                 input_originallabel[y][x] = 0
        #     print(np.unique(input_originallabel))
        #     cv2.imwrite('testtttt.png', input_originallabel)
        # elif len(np.unique(input_originallabel)) < 11:
        #     print("Defected image detected (Less labels):" + f"{i}.png")
        #     defected = f"{i}_.png"
        #     print(np.unique(input_originallabel))
        #     path_to_mask = '/home/eric/Desktop/edit8label'

        #     # 25->9 32->10
        #     path_mask_abs = os.path.join(path_to_mask, defected)
        #     mask = cv2.imread(path_mask_abs, cv2.IMREAD_GRAYSCALE)
        #     for y in range(256):
        #         for x in range (256):
        #             if (input_originallabel[y][x] == 0) and (mask[y][x] == 25):
        #                 input_originallabel[y][x] = 9
        #             elif (input_originallabel[y][x] == 0) and (mask[y][x] == 32):
        #                 input_originallabel[y][x] = 10
        #     print(np.unique(input_originallabel))
        #     label_fixed_path = os.path.join(path_to_mask, f"{i}_fixed.png")
        #     cv2.imwrite(label_fixed_path, input_originallabel)
                    
        label_dataset.append(input_originallabel)

    print("Number of label images:", len(label_dataset))

    for i in input_number:
        # changed this part from 100 to 256
        dataset_img = np.zeros((256, 256, feature_num), dtype=np.float32)

        for m in range(feature_num):
            feature_img_path = os.path.join(feature_dir, str(i), f"{inputfeature_list[m]}.png")
            input_featureimg = cv2.imread(feature_img_path, cv2.IMREAD_GRAYSCALE)
            dataset_img[:, :, m] = input_featureimg

        arrDataset.append(dataset_img)

    arrDataset = np.array(arrDataset)
    print("dataset shape ", arrDataset.shape)
    print("label shape ", np.shape(label_dataset))
    print("end create now")

    return arrDataset, label_dataset


In [13]:
#Augmentation function
# need to configure further to make it receive the new shape of the label images

import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

data_dir = str(root_dir / "img_1006t/mcOriginal")
feature_dir = str(root_dir / "img_1006t/mcFeature") 
labeled_dir = str(root_dir / "img_1006t/labeled")


test_data_dir = str(root_dir / "img_1006t/augMCOriginal/30aug") 
test_labeled_dir = str(root_dir / "img_1006t/augMCLabel/30aug") 
test_feature_dir = str(root_dir / "img_1006t/augMCFeature/30aug")

## 重み計算なし
def ImageAug(input_number):
    print("in aug now")
    label_dataset = []
    arrDataset = []
    for i in input_number:
        label_path = os.path.join(labeled_dir, f"{i}.png")
        input_originallabel = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        label_dataset.append(input_originallabel)


    print("Number of label images:", len(label_dataset))
    
    #means x augment
    augmentation_scale = 30
    
    seq = iaa.Sequential([
    #iaa.Fliplr(0.5),  # horizontal flip with 50% probability
    #iaa.Flipud(0.5),  # vertical flip with 50% probability
    iaa.Affine(
        rotate=(-10, 10),  # rotate by -10 to +10 degrees
        scale=(0.9, 1.1),  # scale by 90% to 110%
        translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)}  # translate by -5% to +5%
    ),
    #iaa.ElasticTransformation(alpha = 3, sigma = 5),  # elastic deformation
    #iaa.Multiply((0.8, 1.2)),  # change brightness
    #iaa.LinearContrast((0.75, 1.5))  # change contrast
])

    
    arr_labelDataset = []
    for i in input_number:
        print(i)
        original_img_path = os.path.join(data_dir, str(i) + ".png")
        input_original_img = cv2.imread(original_img_path, cv2.IMREAD_GRAYSCALE)
        for k in range(augmentation_scale):
            augmented = seq(image = input_original_img, segmentation_maps = SegmentationMapsOnImage(label_dataset[input_number.index(i)], shape = label_dataset[input_number.index(i)].shape))
            arrDataset.append(np.array(augmented[0]))
            arr_labelDataset.append(np.array(augmented[1].get_arr()))
            
        #arrDataset = np.array(arrDataset)
        #arr_labelDataset = np.array(arr_labelDataset)
        for c, aug_data, aug_label in zip(range(1,augmentation_scale + 1), arrDataset, arr_labelDataset):
            path1 = os.path.join(test_data_dir, (i + '-' + str(c)+'.png'))
            path2 = os.path.join(test_labeled_dir, (i + '-' + str(c)+'.png'))
            #print(path1)
            cv2.imwrite(path1, aug_data)
            cv2.imwrite(path2, aug_label)
        
        arrDataset.clear()
        arr_labelDataset.clear()
print(file_names_with_prefix(data_dir, ''))
ImageAug(file_names_with_prefix(data_dir, ''))


['N1-1', 'N1-2', 'N1-3', 'N1-4', 'N1-5', 'N1-6', 'N1-7', 'N1-8', 'N1-9', 'N1-10', 'N2-1', 'N2-2', 'N2-3', 'N2-4', 'N2-5', 'N2-6', 'N2-7', 'N2-8', 'N2-9', 'N2-10', 'N3-1', 'N3-2', 'N3-3', 'N3-4', 'N3-5', 'N3-6', 'N3-7', 'N3-8', 'N3-9', 'N3-10', 'N4-1', 'N4-2', 'N4-3', 'N4-4', 'N4-5', 'N4-6', 'N4-7', 'N4-8', 'N4-9', 'N5-1', 'N5-2', 'N5-3', 'N5-4', 'N5-5', 'N5-6']
in aug now
Number of label images: 45
N1-1
N1-2
N1-3
N1-4
N1-5
N1-6
N1-7
N1-8
N1-9
N1-10
N2-1
N2-2
N2-3
N2-4
N2-5
N2-6
N2-7
N2-8
N2-9
N2-10
N3-1
N3-2
N3-3
N3-4
N3-5
N3-6
N3-7
N3-8
N3-9
N3-10
N4-1
N4-2
N4-3
N4-4
N4-5
N4-6
N4-7
N4-8
N4-9
N5-1
N5-2
N5-3
N5-4
N5-5
N5-6


In [14]:
#############################
##Custom Function generates feature img
#############################

import os
import numpy as np
from PIL import Image

data_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/augMCOriginal/30aug")
feature_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/augMCFeature/30aug") 
labeled_dir = str(root_dir / "img_1006t/MC3x3(108_3out)/augMCLabel/30aug")

def read_images_from_directory(dir_path):
    IMAGE_SIZE = 256


    """
    Reads all image files from a directory and returns a list of images as NumPy arrays.
    
    :param dir_path: Path to the directory containing the images.
    :param image_size: Size of the images (assumes square images).
    
    :return: img_list: List of images as NumPy arrays.
    """
    img_list = []
    fn_without_list = []

    
    # Iterate through all files in the directory
    for filename in os.listdir(dir_path):
        # Check if the file is an image file (you can add more extensions if needed)
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            # Construct the full path to the image
            img_path = os.path.join(dir_path, filename)
            
            # Open the image and resize it to the desired size
            img = Image.open(img_path).convert('L')
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            
            # Convert the image to a NumPy array 
            img_array = np.asarray(img, dtype=np.float64)
            
            # Append the image array to the list
            img_list.append(img_array)
            # Gets the file name without extension for creating dir 
            filename_without_extension = os.path.splitext(filename)[0]
            fn_without_list.append(filename_without_extension)
    
    return img_list, fn_without_list

img_list, fn_without_list = read_images_from_directory(data_dir)
print(len(fn_without_list))
#print(fn_without_list[0])

counter = 0
for fn_wthout, img in zip(fn_without_list, img_list):
    #print("File that is being processed at the moment: " + str(fn_wthout))
    if os.path.exists(os.path.join(feature_dir, fn_wthout)):
        files = os.listdir(os.path.join(feature_dir, fn_wthout))
        png_files = [file for file in files if file.lower().endswith('.png')]
        if png_files:
            print(fn_wthout + ' already generated dataset, skipping...')
            continue
    WriteFile.make_folder(feature_dir, '/' + fn_wthout)
    fn_feature_list = []
    filtered_img_list = []
    for feature in InputFeature:
        #print('Processed feature now:' + str(feature))
        filtered_img= Filtering.single_image(img, feature)
        filtered_img_list.append(filtered_img)
        fn_feature_list.append(str(feature) + '.png')
    WriteFile.save_images(os.path.join(feature_dir,fn_wthout), fn_feature_list, filtered_img_list)
    counter += 1 
    print(str(counter) + ": " + fn_wthout)


1350
1: N5-4-16
2: N2-9-11
3: N5-5-23
4: N3-2-18
5: N4-1-30
6: N4-8-4
7: N3-9-1
8: N1-2-4
9: N4-9-1
10: N1-4-19
11: N1-8-2
12: N2-1-6
13: N2-1-2
14: N1-3-16
15: N1-1-2
16: N5-5-25
17: N2-3-2
18: N1-1-15
19: N1-6-9
20: N2-8-16
21: N4-4-5
22: N5-1-3
23: N1-7-5
24: N1-8-15
25: N4-8-30
26: N1-4-30
27: N5-5-27
28: N2-9-18
29: N3-9-28
30: N5-4-12
31: N1-1-18
32: N3-8-4
33: N4-4-6
34: N5-5-15
35: N2-3-22
36: N2-7-27
37: N3-9-6
38: N5-1-7
39: N1-1-10
40: N1-10-22
41: N5-2-13
42: N1-6-28
43: N2-9-20
44: N1-4-24
45: N5-2-5
46: N3-2-5
47: N2-9-9
48: N2-10-27
49: N4-3-16
50: N3-7-16
51: N5-6-4
52: N1-4-26
53: N4-4-14
54: N2-7-29
55: N2-2-5
56: N1-6-3
57: N2-8-3
58: N3-3-12
59: N3-4-11
60: N5-6-11
61: N3-9-22
62: N4-8-11
63: N3-8-20
64: N1-2-6
65: N2-8-29
66: N2-1-1
67: N5-2-11
68: N5-6-15
69: N3-3-6
70: N5-6-3
71: N1-2-21
72: N4-5-21
73: N3-3-19
74: N4-1-26
75: N2-2-3
76: N5-2-15
77: N3-4-6
78: N1-4-12
79: N1-4-18
80: N2-10-16
81: N4-8-7
82: N5-2-4
83: N2-9-8
84: N3-8-29
85: N1-2-7
86: N3-7-19
87:

/home/eric/Documents/cervicalResearchIIP/MCtool/RFilter.py:882: RuntimeWarning: divide by zero encountered in log
  magnitude_spectrum = 20 * np.log(np.abs(fshift))


1023: N5-4-4
1024: N2-9-24
1025: N4-2-18
1026: N2-2-2
1027: N1-10-12
1028: N3-1-16
1029: N3-2-13
1030: N1-7-4
1031: N2-2-29
1032: N1-8-30
1033: N2-3-27
1034: N2-4-21
1035: N1-1-16
1036: N1-8-1
1037: N3-10-3
1038: N1-4-4
1039: N2-10-15
1040: N1-4-11
1041: N4-9-20
1042: N5-6-27
1043: N2-6-23
1044: N2-10-20
1045: N2-7-17
1046: N4-7-24
1047: N5-3-20
1048: N1-2-29
1049: N4-6-14
1050: N5-4-15
1051: N4-8-16
1052: N1-5-27
1053: N5-5-26
1054: N4-5-13
1055: N2-10-5
1056: N4-7-1
1057: N3-7-10
1058: N3-2-20
1059: N4-7-25
1060: N1-10-17
1061: N4-6-3
1062: N2-9-27
1063: N1-9-7
1064: N4-4-20
1065: N1-1-20
1066: N3-2-23
1067: N2-3-30
1068: N4-9-15
1069: N2-4-26
1070: N2-3-21
1071: N2-6-3
1072: N2-10-22
1073: N5-2-1
1074: N1-8-14
1075: N1-2-11
1076: N1-2-24
1077: N3-5-1
1078: N1-3-25
1079: N3-6-13
1080: N2-7-15
1081: N1-7-1
1082: N2-10-6
1083: N5-4-23
1084: N4-8-28
1085: N2-10-13
1086: N2-6-24
1087: N3-2-22
1088: N3-4-15
1089: N2-7-21
1090: N3-1-28
1091: N3-2-30
1092: N2-10-10
1093: N4-3-11
1094: N1-4-

In [32]:
# fn_wthout = 'N4-9-1'
# print(os.path.join(feature_dir, fn_wthout))
# path = os.path.join(feature_dir, fn_wthout)
# if(os.path.exists(path)):
#     files = os.listdir(os.path.join(feature_dir, fn_wthout))
#     print(files)
#     png_files = [file for file in files if file.lower().endswith('.png')]
#     print(png_files)
#     png_files.clear()
#     if(png_files):
#         print('yes')

/home/eric/Documents/cervicalResearchIIP/img_1006t/testFeature/N4-9-1
['TOP3.png', 'MAX2.png', 'MRL2.png', 'UNS2.png', 'MIN2.png', 'SBL1.png', 'MRL3.png', 'FOU4.png', 'SBL2.png', 'NML2.png', 'ERO2.png', 'RIC_.png', 'BLT1.png', 'SBLM.png', 'LPL2.png', 'PRE4.png', 'GRY_.png', 'NGP_.png', 'FOU1.png', 'FOU3.png', 'NML3.png', 'POS1.png', 'LBP3.png', 'ERO5.png', 'OOO_.png', 'TOP2.png', 'MED1.png', 'OPN2.png', 'SBLY.png', 'MIN3.png', 'BTM3.png', 'ROB1.png', 'PRE2.png', 'MAX3.png', 'BLT2.png', 'FOU2.png', 'OPN3.png', 'BTM4.png', 'ERO3.png', 'HOM_.png', 'ETC1.png', 'MEA1.png', 'ROB3.png', 'SCH2.png', 'CAN2.png', 'UNS3.png', 'MAX1.png', 'CLO3.png', 'KNN1.png', 'SCH3.png', 'CAN3.png', 'POS3.png', 'CLO4.png', 'CLO1.png', 'UNS5.png', 'SCH1.png', 'ERO6.png', 'CLO5.png', 'STC1.png', 'LPL1.png', 'EMB1.png', 'HGF_.png', 'MIN4.png', 'SCH4.png', 'MRG3.png', 'LBP1.png', 'MAX4.png', 'SOL_.png', 'ROB4.png', 'EMB3.png', 'TOP4.png', 'STC2.png', 'BTM1.png', 'UNS4.png', 'TOP1.png', 'NML1.png', 'SBLX.png', 'ROB2